## Weather and Climate Data Exercise One - "Process"
This is the question: http://giub-torrent.unibe.ch/weather-simulation/#process

*Convert the pressure readings to gravity-corrected station pressure (in hPa), temperature-adjusted to 0 °C (the present-day standard). If you want you can also reduce pressure to mean sea level*

### Notes on the data
The file was converted to .txt format and saved with the following modifications:
- text set to lowercase
- some column headers renamed
- formulas here: http://giub-torrent.unibe.ch/weather-simulation/data/ClimateData_Process_Formula.pdf

In [1]:
# import packages and data

import pandas as pd
import os
import sys
import datetime as dt
import json
import csv
import math

In [2]:
mypd = pd.read_csv("ClimateData_Process_Exercise.csv")

In [3]:
# First, convert the temperature and distance measurements to modern units

# constant in paris inches to millimeters
pinch = 27.07
tream = 0.8

# convert pressure heights at sunrise and sunset from paris inches to millimeters 
mypd["mme"] = mypd["pressure_e"]*pinch
mypd["mmt"] = mypd["pressure_t"]*pinch

# convert temperature measurements from reamur to celsius
mypd["tempe"] = mypd["temperature_e"]*tream
mypd["tempt"] = mypd["temperature_t"]*tream

In [4]:
# Second, convert the pressure reading at 10° Reamur to 0° Celsius

#convert Reamur ° to Celsius ° and store in constant
temp = 10/0.8

# define lambda function for conversion according to Mercury's coefficient of thermal expansion at 0°C

def my_func(x):
    data = (1-0.000182*temp)*x
    return data

# call lambda function on relevant columns
mypd.mme=mypd.mme.apply(lambda x: my_func(x))
mypd.mmt=mypd.mmt.apply(lambda x: my_func(x))

In [5]:
# Third, convert the pressure to modern units hPa

# define constants (standard gravity and mercury'y density at 0 degrees celsius respectively)
grav = 9.80665
ro = 13595.1

# define lambda function for conversion to moderun units hPa
def my_lambda(x):
    data = ro*grav*x*0.00001
    return data

# call lambda function on relevant columns
mypd.mme=mypd.mme.apply(lambda x: my_lambda(x))
mypd.mme = mypd.mme/10
mypd.mmt=mypd.mmt.apply(lambda x: my_lambda(x))
mypd.mmt = mypd.mmt/10

In [6]:
# Fourth, correct gravity changes from latitude and altitude
# define constants altitude and latitude
alt = 776
lat = 0.827

altc = 776*0.000003086
step1=0.0026442*math.cos(2*0.827)
step2 =0.0000058*math.cos(2*0.827)*math.cos(2*0.827)
constant = 9.80620*(1-step1-step2)-altc

def my_newlamb(x):
    data = x*constant
    return data

mypd.mme=mypd.mme.apply(lambda x: my_newlamb(x))
mypd.mmt=mypd.mmt.apply(lambda x: my_newlamb(x))

In [7]:
# Fifth correct for mean sea level (leave as review exercise)

In [8]:
#export final data set:
mypd.to_csv(path_or_buf = "exercise1.csv", index = False)

In [9]:
mypd.columns

Index(['year', 'month', 'day', 'pressure_e', 'temperature_e', 'wind_e',
       'pressure_t', 'temperature_t', 'wind_t', 'morning_s', 'afternoon_s',
       'morning_r', 'afternoon_r', 'mme', 'mmt', 'tempe', 'tempt'],
      dtype='object')

In [10]:
mypd.head()

,year,month,day,pressure_e,temperature_e,wind_e,pressure_t,temperature_t,wind_t,morning_s,afternoon_s,morning_r,afternoon_r,mme,mmt,tempe,tempt
0,1813,1,1,26.08,3.2,s,26.09,4.0,s,0,0,0,0,920.873316,921.226411,2.56,3.20
1,1813,1,2,26.11,3.2,s,26.11,3.0,s,0,0,0,0,921.932603,921.932603,2.56,2.40
2,1813,1,3,26.15,2.6,n,26.20,3.2,n,1,0,0,0,923.344985,925.110463,2.08,2.56
3,1813,1,4,26.22,2.8,n,26.21,3.8,n,0,0,0,0,925.816654,925.463559,2.24,3.04
4,1813,1,5,26.17,2.8,s,26.17,3.0,so,0,0,0,0,924.051176,924.051176,2.24,2.40
